In [217]:
from gabor_feature import get_feature_vector
import cv2
import os
from collections import *
import numpy as np
from sklearn.cluster import KMeans

_k = 4
_n = 4
_w0 = 0.13
_folder_views = "images/"
_n_clusters = 10

def getAllViews():
    l = os.listdir(_folder_views)
    all_views = []
    for i in l:
        all_views.append(_folder_views + i)
    return all_views

In [97]:
img = cv2.imread("sketches/888.png", cv2.COLOR_RGB2GRAY)
basic_views = ["images/m508_outfile_10.jpg"]

In [152]:
import tqdm

In [191]:
features = []
features_per_objectview = []
names = []
basic_views = getAllViews()
for basic_view in tqdm.tqdm(basic_views[:11]):
    sketch = cv2.Canny(cv2.imread(basic_view, 0), 100, 200)
    if np.sum(sketch) == 0:
        continue
    features_objectview = [get_feature_vector(sketch, k=_k, n=_n, w0=_w0), get_feature_vector(sketch, k=_k, n=_n, w0=_w0*5)]
    features.extend(features_objectview )
    features_per_objectview.append(features_objectview)
    names.append(basic_view)
kmeans = KMeans(n_clusters=_n_clusters, random_state=0).fit(features)

100%|██████████| 11/11 [00:00<00:00, 18.75it/s]


In [300]:
def createHistogram(centers):
    h = Counter()
    for i in centers:
        h[i] += 1
    h = dict(sorted(h.items()))
    return h

def createHistograms(kmeans, features_per_objectview):
    histograms = []
    for feature_objectview in features_per_objectview:
        histograms.append(createHistogram(kmeans.predict(feature_objectview)))
    histograms.sort(key=lambda x: [(i, x[i]) for i in x.keys()])
    return histograms

def processSketchHistogram(h,N,f):
    hi_sum = np.sum(list(h.values()))
    for j in h.keys():
        h[j] = h[j]/hi_sum*np.log(N/f[j])
    h_sum = np.sum([i**2 for i in h.values()])
    for j in h.keys():
        h[j] = h[j]/h_sum
    print(h)

In [301]:
print(histogram_for_sketch)
processSketchHistogram(histogram_for_sketch, N, f)

{0: 1.3136954291957648, 3: 0.04606891079797685, 8: 0.3355943922291999, 9: 0.3355943922291999}
{0: 0.6047513817734528, 3: 0.006973296396863222, 8: 0.10983149046876829, 9: 0.10983149046876829}


In [264]:
histograms = createHistograms(kmeans, features_per_objectview)

In [265]:
histograms

[{0: 1, 3: 1},
 {1: 1, 8: 1},
 {1: 1, 8: 1},
 {2: 2},
 {3: 2},
 {3: 2},
 {4: 1, 7: 1},
 {5: 2},
 {6: 2},
 {7: 2},
 {9: 2}]

In [266]:
N = len(histograms)
f = np.zeros(kmeans.n_clusters)
for h in histograms:
    for j in h.keys():
        f[j] = f[j] + h[j]

In [267]:
f

array([1., 2., 2., 5., 1., 2., 2., 3., 2., 2.])

In [283]:
example = np.random.random([6, _n**2*_k])

In [284]:
histogram_for_sketch = createHistogram(kmeans.predict(example))
    

In [280]:
histogram_for_sketch

[{1: 3, 3: 2, 8: 1}]

In [161]:
histograms

[{0: 1, 2: 1, 5: 2, 7: 2},
 {0: 1, 3: 1, 5: 1, 7: 1, 8: 2},
 {2: 1, 3: 2, 5: 1, 7: 2},
 {2: 2, 6: 1, 7: 2, 8: 1},
 {2: 2, 7: 3, 8: 1},
 {3: 1, 4: 1, 5: 1, 6: 2, 7: 1},
 {5: 1, 6: 2, 7: 3},
 {5: 2, 6: 3, 7: 1},
 {5: 2, 7: 3, 8: 1}]

In [134]:
class RangedHistograms():
    def __init__(self, length):
        self.histograms = []
        self.max_len = length
        
    def add(self, h):
        if len(self.histograms) == 0:
            self.histograms.append(h)
        else:
            for i in list(h.keys):
                print(i)
            
    def show(self):
        print(self.histograms)
            
t = RangedHistograms(10)
t.add(histograms[0])
t.show()
t.add(histograms[1])

[{0: 3, 6: 1, 8: 1, 1: 1}]


TypeError: 'builtin_function_or_method' object is not iterable